In [2]:
import numpy as np
import pandas as pd
import regex as re
from multiprocessing.pool import ThreadPool
import random
from scraper import StockInfo
from dcfModel import DCF

In [3]:
ticker = "NVDA"
operatingMargin = np.array([35, 35, 37, 38, 39, 40, 40, 40, 40, 40, 40, 40]) / 100

RevenueGrowthRate = np.array([15, 15, 15, 15, 15, 12.72, 10.44, 8.16, 5.88, 3.60]) / 100
Costofcapital = np.array([12.21, 12.21, 12.21, 12.21, 12.21, 11.54, 10.86, 10.19, 9.52, 8.85]) / 100
TaxRate = np.array([10, 10, 10, 10, 10, 10, 13, 16, 19, 22, 25, 25]) / 100
salesToCapital = np.array([1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15])
forecastYears = 10
baseRevenue = 14028

In [5]:
valuation = DCF(ticker, operatingMargin, RevenueGrowthRate, TaxRate, Costofcapital, salesToCapital, baseRevenue, forecastYears)
valuation.make_Value()

111521.93313263974

In [6]:
tick = StockInfo("ORCL")

In [8]:
cash = tick.get_FreeCashFlow()
cash

,Breakdown,ttm,5/31/2023,5/31/2022,5/31/2021,5/31/2020
0,Operating Cash Flow,"17,745,000","17,165,000","9,539,000","15,887,000","13,139,000"
1,Investing Cash Flow,"-8,610,000","-36,484,000","11,220,000","-13,098,000","9,843,000"
2,Financing Cash Flow,"-7,928,000","7,910,000","-29,126,000","-10,378,000","-6,132,000"
3,End Cash Position,"11,655,000","9,765,000","21,383,000","30,098,000","37,239,000"
4,Income Tax Paid Supplemental Data,"3,009,000","2,567,000","3,189,000","3,218,000",NaN
5,Interest Paid Supplemental Data,"3,250,000","2,735,000","2,408,000","1,972,000",NaN
6,Capital Expenditure,"-8,290,000","-8,695,000","-4,511,000","-2,135,000","-1,564,000"
7,Issuance of Capital Stock,"985,000","1,192,000","482,000","1,786,000","1,588,000"
8,Issuance of Debt,"13,948,000","33,994,000",0,"14,934,000","19,888,000"
9,Repayment of Debt,"-16,729,000","-21,050,000","-8,250,000","-2,631,000","-4,500,000"
